In [20]:
'''
Урок 4. Парсинг HTML. XPath
Написать приложение, которое собирает основные новости с сайтов mail.ru, lenta.ru.
Для парсинга использовать xpath. Структура данных должна содержать:
* название источника,
* наименование новости,
* ссылку на новость,
* дата публикации
'''

import requests
from lxml import html
import pandas as pd
import datetime

headers = {'Accept': '*/*',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36\
            (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}


In [21]:
#импортируем новости из 'lenta.ru'

#можно конечно считать это читерством,
#но в мобильной версии ленты структура сайта гораздо более простая
#поэтому возьмём новости из 'm.lenta.ru'

#формируем списки
list_data_source = []
list_news_link = []
list_public_date = []
list_news_text = []

link = 'https://m.lenta.ru/'
request = requests.get(link, headers=headers)
root = html.fromstring(request.text)

#Ссылки на новости
#result_list = root.xpath('//li[@class="b-list-item b-list-item_news"]//a[starts-with(@href, "/news")]/@href')

#Даты публикации
#result_list = root.xpath('//li[@class="b-list-item b-list-item_news"]//a[starts-with(@href, "/news")]/span/time/@title')

#Текст новости
#result_list = root.xpath('//li[@class="b-list-item b-list-item_news"]//a[starts-with(@href, "/news")]/span[@class="b-list-item__title"]/text()')

result_list = root.xpath('//li[@class="b-list-item b-list-item_news"]//a[starts-with(@href, "/news")]')

#составим словарь для преобразования месяца
month_dict = {'января':1,'февраля':2,'марта':3,'апреля':4,'мая':5,'июня':6,'июля':7,
              'августа':8,'сентября':9,'октября':10,'ноября':11,'декабря':12}

#наполняем списки данными
for i in result_list:
    
    #Источник данных
    data_source = 'lenta.ru'
    list_data_source.append(data_source)
    
    #Ссылка на новость
    try:
        news_link = 'https://lenta.ru/' + i.xpath('./@href')[0]
    except Exception as e:
        print(str(i) + ': '+ str(e))
        news_link = None
    list_news_link.append(news_link)
    
    #Текст новости
    try:
        news_text = i.xpath('./span[@class="b-list-item__title"]/text()')[0]
    except Exception as e:
        print(news_link + ' : '+ str(e))
        news_text = None
    list_news_text.append(news_text)

    #Дата публикации    
    try:
        public_date_string = i.xpath('./span/time/@title')[0]
        date = public_date_string.split()
        public_date =  datetime.date(int(date[2]), month_dict.get(date[1]), int(date[0]))
    except Exception as e:
        #Перейдём на саму страницу новости:
        link = news_link
        request = requests.get(link, headers=headers)
        root = html.fromstring(request.text)
        public_date = root.xpath('//div[@class ="b-topic__info"]/time[@ class ="g-date"]/@datetime')[0].split('T')[0]
    list_public_date.append(public_date)
    
news_lenta = pd.DataFrame({'data_source':list_data_source,
                     'news_link':list_news_link,
                     'news_text':list_news_text,
                     'public_date':list_public_date})    
news_lenta.head(10)    

,data_source,news_link,news_text,public_date
0,lenta.ru,https://lenta.ru//news/2019/10/25/person/,Раскрыта личность устроившего бойню в Забайкал...,2019-10-25
1,lenta.ru,https://lenta.ru//news/2019/10/25/profitablypr...,Пассажирка нашла хитрый способ не платить за б...,2019-10-25
2,lenta.ru,https://lenta.ru//news/2019/10/25/dedovshina/,Причиной расстрела солдат в российской воинско...,2019-10-25
3,lenta.ru,https://lenta.ru//news/2019/10/25/butina/,Бутину освободили из федеральной тюрьмы,2019-10-25
4,lenta.ru,https://lenta.ru//news/2019/10/25/ssj_out/,Норвежская компания опровергла покупку SSJ-100,2019-10-25
5,lenta.ru,https://lenta.ru//news/2019/10/25/weapon/,В России признали преимущество УССР перед РСФС...,2019-10-25
6,lenta.ru,https://lenta.ru//news/2019/10/25/ssora/,Шойгу рассказал о попытках Запада поссорить Ро...,2019-10-25
7,lenta.ru,https://lenta.ru//news/2019/10/25/drug/,Россиянин уколол прохожую шприцем с наркотиком,2019-10-25
8,lenta.ru,https://lenta.ru//news/2019/10/25/taco/,231-сантиметровый баскетболист ударился о низк...,2019-10-25
9,lenta.ru,https://lenta.ru//news/2019/10/25/soleimani/,Командира элитного иранского спецназа заметили...,2019-10-25


In [22]:
#импортируем новости из 'mail.ru'

#возьмём новости из 'm.mail.ru' (мобильной версии майла),
#как и из ленты

#формируем списки
list_data_source = []
list_news_link = []
list_public_date = []
list_news_text = []

link = 'https://mail.ru/?from=m'
request = requests.get(link, headers=headers)
root = html.fromstring(request.text)

result_list = root.xpath('//a[@class="list__item"]')  

for i in result_list:
    
    #Источник данных
    data_source = 'mail.ru'
    list_data_source.append(data_source)
    
    #Ссылка на новость
    try:
        news_link = i.xpath('./@href')[0]
    except Exception as e:
        print(str(i) + ': '+ str(e))
        news_link = None
    list_news_link.append(news_link)
    
    #Текст новости
    try:
        news_text = i.xpath('./div/span/text()')[0].replace('\xa0',' ')
    except Exception as e:
        print(news_link + ' : '+ str(e))
        news_text = None
    list_news_text.append(news_text)
    
    #Дата публикации ни в мобильной, ни в основной версии не отображена
    #Перейдём непосредственно на страницу новости:
    #(занимает больше всего времени)
    link = news_link
    request = requests.get(link, headers=headers)
    root = html.fromstring(request.text)
    try:
        public_date = root.xpath('//span[@class="note__text breadcrumbs__text js-ago"]/@datetime\
                                 |//time[@class="note__text breadcrumbs__text js-ago"]/@datetime')[0].split('T')[0]
    except Exception as e:
        print(news_link + ' : '+ str(e))
        public_date = None
    list_public_date.append(public_date)    

news_mail = pd.DataFrame({'data_source':list_data_source,
                     'news_link':list_news_link,
                     'news_text':list_news_text,
                     'public_date':list_public_date})
news_mail.head(10)          


,data_source,news_link,news_text,public_date
0,mail.ru,https://r.mail.ru/n312843897?&test_id=63&rnd=5...,В Забайкалье военный застрелил восьмерых сослу...,2019-10-25
1,mail.ru,https://r.mail.ru/n312849705?&test_id=63&rnd=5...,Марию Бутину освободили из тюрьмы в США для де...,2019-10-25
2,mail.ru,https://r.mail.ru/n312826264?&test_id=63&rnd=5...,Минфин и ЦБ согласовали проект «второй пенсии»,2019-10-25
3,mail.ru,https://r.mail.ru/n312844422?&test_id=63&rnd=5...,Проживающую 20 лет в школе учительницу обеспеч...,2019-10-25
4,mail.ru,https://r.mail.ru/n312846148?&test_id=63&rnd=5...,Китайцы возродили заброшенную русскую деревню,2019-10-25
5,mail.ru,https://r.mail.ru/n312842714?&test_id=63&rnd=5...,Американка нашла в куртке из секонд-хенда огро...,2019-10-25
6,mail.ru,https://r.mail.ru/n312836883?&test_id=63&rnd=5...,Диетолог назвала несовместимые сочетания проду...,2019-10-25
7,mail.ru,https://r.mail.ru/n312828683?&test_id=63&rnd=5...,Улетевшая в Иран птица разорила сибирских ученых,2019-10-25
8,mail.ru,https://r.mail.ru/n312831942?&test_id=63&rnd=5...,Дым от пожара в районе имени Лазо принесло в Х...,2019-10-25
9,mail.ru,https://r.mail.ru/n312831460?&test_id=63&rnd=5...,В Хабаровске недействительны около 20 проценто...,2019-10-25


In [23]:
news_lenta_mail = pd.concat([news_lenta, news_mail]) 
news_lenta_mail

,data_source,news_link,news_text,public_date
0,lenta.ru,https://lenta.ru//news/2019/10/25/person/,Раскрыта личность устроившего бойню в Забайкал...,2019-10-25
1,lenta.ru,https://lenta.ru//news/2019/10/25/profitablypr...,Пассажирка нашла хитрый способ не платить за б...,2019-10-25
2,lenta.ru,https://lenta.ru//news/2019/10/25/dedovshina/,Причиной расстрела солдат в российской воинско...,2019-10-25
3,lenta.ru,https://lenta.ru//news/2019/10/25/butina/,Бутину освободили из федеральной тюрьмы,2019-10-25
4,lenta.ru,https://lenta.ru//news/2019/10/25/ssj_out/,Норвежская компания опровергла покупку SSJ-100,2019-10-25
5,lenta.ru,https://lenta.ru//news/2019/10/25/weapon/,В России признали преимущество УССР перед РСФС...,2019-10-25
6,lenta.ru,https://lenta.ru//news/2019/10/25/ssora/,Шойгу рассказал о попытках Запада поссорить Ро...,2019-10-25
7,lenta.ru,https://lenta.ru//news/2019/10/25/drug/,Россиянин уколол прохожую шприцем с наркотиком,2019-10-25
8,lenta.ru,https://lenta.ru//news/2019/10/25/taco/,231-сантиметровый баскетболист ударился о низк...,2019-10-25
9,lenta.ru,https://lenta.ru//news/2019/10/25/soleimani/,Командира элитного иранского спецназа заметили...,2019-10-25
